# Using scvi_de

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

import scanpy as sc
import muon as mu

/mnt/wsl/PHYSICALDRIVE0p1/workspace/scvi_de/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


if scvi_de hasn't been installed, uncomment and run the following cell

In [3]:
from scvi_de import scvi_de, process_mudata, is_integer_array, create_model

In [4]:
submudata = mu.read_h5mu("subsubset_bcells.h5mu")

/mnt/wsl/PHYSICALDRIVE0p1/workspace/scvi_de/.venv/lib/python3.10/site-packages/anndata/_core/anndata.py:430: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/mnt/wsl/PHYSICALDRIVE0p1/workspace/scvi_de/.venv/lib/python3.10/site-packages/anndata/_core/anndata.py:430: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(


In [5]:
for _ in ('rna', 'prot'):
    if 'rank_genes_groups' in submudata[_].uns:
        del(submudata[_].uns['rank_genes_groups'])

In [6]:
for _ in submudata.mod:
    submudata[_].X = submudata[_].raw[submudata[_].obs_names,submudata[_].var_names].X.toarray().copy()

In [7]:
import scvi

In [8]:
scvi.model.TOTALVI.setup_mudata(mdata=submudata, batch_key="batch", modalities={"rna_layer": "rna", "protein_layer": "prot"})

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


INFO     Found batches with missing protein expression                                                             


In [9]:
model = scvi.model.TOTALVI(submudata, protein_dispersion="protein-batch", gene_likelihood="zinb")

INFO     Computing empirical prior initialization for protein background.                                          


/tmp/ipykernel_29168/677091434.py:1: UserWarning: Some proteins have all 0 counts in some batches. These proteins will be treated as missing measurements; however, this can occur due to experimental design/biology. Reinitialize the model with `override_missing_proteins=True`,to override this behavior.
  model = scvi.model.TOTALVI(submudata, protein_dispersion="protein-batch", gene_likelihood="zinb")


In [10]:
model.train(
    check_val_every_n_epoch=1,
    max_epochs=400,
    early_stopping=True,
    early_stopping_patience=20,
    early_stopping_monitor="elbo_validation",
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/mnt/wsl/PHYSICALDRIVE0p1/workspace/scvi_de/.venv/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "
/mnt/wsl/PHYSICALDRIVE0p1/workspace/scvi_de/.venv/lib/python3.10/site-packages/lightning/pytorch/core/optimizer.py:314: The lr scheduler dict contains the key(s) ['monitor'], but the keys will be ignored. You need to call `lr_scheduler.step()` manually in manual optimization.
/mnt/wsl/PHYSICALDRIVE0p1/workspace/scvi_de/.venv/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to 

Epoch 39/400:  10%|████████████▊                                                                                                                      | 39/400 [00:40<06:16,  1.04s/it, v_num=1, train_loss_step=3.75e+3, train_loss_epoch=4.02e+3]
Monitored metric elbo_validation did not improve in the last 20 records. Best score: 7999.908. Signaling Trainer to stop.


In [11]:
degs = model.differential_expression(groupby="leiden_wnn_0.5", batch_correction=True,)

DE...: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [01:24<00:00, 12.12s/it]


In [12]:
degs

,proba_de,proba_not_de,bayes_factor,scale1,scale2,pseudocounts,delta,lfc_mean,lfc_median,lfc_std,...,raw_mean1,raw_mean2,non_zeros_proportion1,non_zeros_proportion2,raw_normalized_mean1,raw_normalized_mean2,is_de_fdr_0.05,comparison,group1,group2
Hu.CD62P,0.888756,0.111244,2.078092,2.358508,2.272243,0.0,0.25,0.247274,0.301492,1.636038,...,4.301724,3.007229,0.758621,0.662651,NaN,NaN,False,0 vs Rest,0,Rest
Hu.HLA.DR,0.881953,0.118047,2.011054,7.464637,6.169066,0.0,0.25,0.370331,0.405734,1.654876,...,29.991379,11.322891,0.978448,0.922892,NaN,NaN,False,0 vs Rest,0,Rest
Hu.CD1c,0.874950,0.125050,1.945453,2.724365,2.327896,0.0,0.25,0.342799,0.368826,1.497016,...,6.202586,4.534940,0.814655,0.672289,NaN,NaN,False,0 vs Rest,0,Rest
Hu.CD40,0.873549,0.126451,1.932713,5.110736,4.155727,0.0,0.25,0.354050,0.355344,1.598054,...,15.840517,7.245783,0.965517,0.889157,NaN,NaN,False,0 vs Rest,0,Rest
Hu.CD31,0.873349,0.126651,1.930903,3.030804,2.180837,0.0,0.25,0.513343,0.553572,1.438041,...,8.297414,3.209639,0.948276,0.785542,NaN,NaN,False,0 vs Rest,0,Rest
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PCNP,0.156463,0.843537,-1.684787,0.000117,0.000113,0.0,0.25,0.055220,0.056949,0.170088,...,0.333333,0.358814,0.333333,0.287051,1.866352,1.140848,False,6 vs Rest,6,Rest
TUBGCP2,0.150860,0.849140,-1.727869,0.000086,0.000086,0.0,0.25,-0.002826,-0.001178,0.176672,...,0.166667,0.157566,0.166667,0.137285,0.865351,0.467735,False,6 vs Rest,6,Rest
RNPS1,0.127851,0.872149,-1.920093,0.000140,0.000137,0.0,0.25,0.030187,0.025493,0.160384,...,0.000000,0.474259,0.000000,0.351014,0.000000,1.490342,False,6 vs Rest,6,Rest
ACTR2,0.123449,0.876551,-1.960163,0.000160,0.000157,0.0,0.25,0.028264,0.032110,0.163708,...,0.833333,0.630265,0.666667,0.422777,3.339299,1.932439,False,6 vs Rest,6,Rest


In [14]:
degs["group1"].unique()

array(['0', '1', '2', '3', '4', '5', '6'], dtype=object)

In [19]:
deg_dict = {i: degs.loc[degs["group1"] == i,:] for i in degs["group1"].unique()}

In [21]:
degs["group1"].nunique()

7